# Is there a corelation between the weather or the days of the week and how much I tend to watch on Netflix?

Here we are going to analyze my Netflix viewing history from a downloaded csv file that lists the title of what I watched and the date of when I watched it. Combingin this with local weather data from an API and the days of the week will allow me to test if:

* I tend to watch more on the weekends
* Watch more when the weather is poor (ie. raining heavily)

To test this I will do a hypothesis test of the frequency of things watched on Fridays, Saturdays, and Sundays ( ie. my classification of the weekend) compared to how much I watched in the week (ie. Monday-Thursday)
>NOTE: I am classifying Friday as the weekend because I believe I probably watched more in the evening on a Friday due to usually not having a responsiblity I need to wake up for the next morning

To do these test and combine these separate pieces of data. But, we first need to set up APIs to get the weather from certain days and the days of the week. 
Since the first instance from the NEtflix Viewing history csv file is dated "7/30/16" we need to import weather data and the days of the week from "7/30/16" to the last recorded day in the viewing hisotry file which is "4/9/2022".
>NOTE: for the weather I am going to just use the weather data from the Sammamish, WA area (where I live) since for the most part I watch netflix at home.

## Code to import all of the data
##### Importing the Weather
Getting the longitude and latitude for Sammamish.
This will allow me to use a spearate API to get the closest weather station ID to Sammamish

In [66]:
import json
import numpy as np
import pandas as pd
import requests

f = open("lat.txt", 'r')
key = f.read()
long_lat_url = "http://www.mapquestapi.com/geocoding/v1/address?key=" + key + "&location=Sammamish"
request_city = requests.get(long_lat_url)
json_city_str = request_city.text
json_city_obj = json.loads(json_city_str)

# getting latitiude and longititude of Sammamish
city_results_list = json_city_obj["results"]
dict = city_results_list[0]
locations_list = dict["locations"]
locations_dict = locations_list[0]
longLat_dict = locations_dict["displayLatLng"]
latitude = longLat_dict["lat"]
longitude = longLat_dict["lng"]

print("Sammamish latitude:", latitude)
print("Sammamish longitude:",longitude)

ovhWlZyIwAjTQ6xOM1iyYhvcW21H3hRk
Sammamish latitude: 47.60553
Sammamish longitude: -122.035555


Getting the weather stations ID using a separate API:

In [67]:
f = open("Weather.txt", 'r')
key = f.read()
headers = {"x-rapidapi-key": key}
url = "https://meteostat.p.rapidapi.com/stations/nearby?lat=" + str(latitude) + "&lon=" + str(longitude) + "&limit=1"
request = requests.get(url = url, headers = headers)

json_station_str = request.text
json_station_obj = json.loads(json_station_str)
station_list = json_station_obj["data"]
station_dict = station_list[0]
station_id = station_dict["id"]
print("Weather Station ID:", station_id)

Weather Station ID: KRNT0


Now finally here we are loading the weather collected from this weather station from during the time period of 7/30/16 to 4/9/2022:

In [68]:
# get daily weather from station 2016-07-30 through 2022-04-09
headers = {"x-rapidapi-key": key}
weather_url = "https://meteostat.p.rapidapi.com/stations/daily?station="+ station_id + "&start=2016-07-30&end=2022-04-09"
request_weather = requests.get(url = weather_url, headers = headers)
json_weather_str = request_weather.text
json_weather_obj = json.loads(json_weather_str)
weather_df = pd.DataFrame(json_weather_obj["data"])
weather_df.set_index("date", inplace = True)
weather_df.to_csv("daily_weather.csv")
print(weather_df)


            tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun
date                                                                     
2016-07-30  19.8  15.6  25.0   0.0  None    NaN  11.2  None  1014.1  None
2016-07-31  18.4  14.4  22.8   0.0  None    NaN   9.1  None     NaN  None
2016-08-01  18.0  12.8  25.0   0.0  None    NaN   6.9  None  1018.2  None
2016-08-02  17.6  15.0  21.7   NaN  None    NaN   NaN  None     NaN  None
2016-08-03  19.2  15.6  23.9   0.0  None    NaN   NaN  None  1024.2  None
...          ...   ...   ...   ...   ...    ...   ...   ...     ...   ...
2022-04-05   7.6   4.0  12.0   0.8  None  172.0  14.7  None  1025.3  None
2022-04-06   8.8   2.0  16.0   0.0  None    6.0   9.7  None  1030.7  None
2022-04-07  14.3   7.0  22.0   0.0  None  354.0   7.7  None  1021.4  None
2022-04-08  10.1   7.0  15.0  10.2  None  194.0  18.2  None  1022.6  None
2022-04-09   5.9   3.5   9.0   5.3  None  150.0  10.8  None  1025.0  None

[2080 rows x 10 columns]


##### Importing the days of the week:
Next we need to load in the days of the week from 2016-07-30 through to 2022-04-09.
To do this we are using the datetime library to create a dataframe with the dates within this parameter and then the corresponding day of the week


In [ ]:
import datetime
date_df = pd.DataFrame()
date_df["Date"] = pd.Series(pd.date_range('2016-7-30', '2022-4-9', freq='D'))
date_df["Day of Week"] = date_df['Date'].dt.day_name()
date_df.set_index("Date", inplace = True)
date_df.to_csv("Days_of_The_Week.csv")
print(date_df)

           Day of Week
Date                  
2016-07-30    Saturday
2016-07-31      Sunday
2016-08-01      Monday
2016-08-02     Tuesday
2016-08-03   Wednesday
...                ...
2022-04-05     Tuesday
2022-04-06   Wednesday
2022-04-07    Thursday
2022-04-08      Friday
2022-04-09    Saturday

[2080 rows x 1 columns]


## Cleaning the data
Now that we have loaded all of the data we need to clean the files in order to accurately compute data and make hypotheses